# Guided Capstone Step 4 - Pre-processing and Training Data Development

### **The Data Science Method**  


1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   **Pre-processing and Training Data Development**

5.  Modeling
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

In this step of the guided capstone, you'll revisit some of the steps you completed to solve questions in step 4. Revisiting these steps will help you get additional practice completing this kind of work.

**<font color='teal'> Start by loading the necessary packages, as you've done in the previous steps. Print out your current working directory to confirm that you are in the correct project directory. </font>**

In [25]:
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

**<font color='teal'>  Load the csv file you created in step 3. Remember, it should be saved inside your data subfolder. Print the first five rows.</font>**

In [26]:
df=pd.read_csv('/resources/springboard/GuidedCapstone/data/step3_output.csv',index_col=False)
df.head(3)

,Unnamed: 0,Unnamed: 0.1,Name,state,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,Cluster
0,2,2,Hilltop Ski Area,Alaska,2090,294,0,0.0,0,0,...,30.0,30.0,150.0,36.0,69.0,30.0,34.0,152.0,30.0,2
1,4,4,Sunrise Park Resort,Arizona,11100,1800,0,0.0,0,1,...,800.0,80.0,115.0,49.0,250.0,74.0,78.0,104.0,80.0,1
2,5,5,Yosemite Ski & Snowboard Area,California,7800,600,0,0.0,0,0,...,88.0,80.0,110.0,84.0,300.0,47.0,47.0,107.0,0.0,1


In [27]:
df_name=df.Name

In [28]:
df=df.drop(columns=['Name', 'Unnamed: 0', 'Unnamed: 0.1'])
df.head(5)

,state,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,double,...,SkiableTerrain_ac,Snow Making_ac,daysOpenLastYear,yearsOpen,averageSnowfall,AdultWeekday,AdultWeekend,projectedDaysOpen,NightSkiing_ac,Cluster
0,Alaska,2090,294,0,0.0,0,0,0,1,0,...,30.0,30.0,150.0,36.0,69.0,30.0,34.00000,152.0,30.0,2
1,Arizona,11100,1800,0,0.0,0,1,2,3,1,...,800.0,80.0,115.0,49.0,250.0,74.0,78.00000,104.0,80.0,1
2,California,7800,600,0,0.0,0,0,0,1,3,...,88.0,80.0,110.0,84.0,300.0,47.0,47.00000,107.0,0.0,1
3,California,7700,500,0,0.0,0,1,1,3,1,...,380.0,200.0,150.0,54.0,400.0,49.0,64.16681,150.0,200.0,1
4,California,8200,1600,0,0.0,0,0,1,2,5,...,862.0,200.0,150.0,69.0,350.0,78.0,78.00000,140.0,0.0,1


## Create dummy features for categorical variables

**<font color='teal'> Create dummy variables for `State`. Add the dummies back to the dataframe and remove the original column for `State`. </font>**

In [29]:
dfo=df.select_dtypes(include=['object']) # select object type columns
df = pd.concat([df.drop(dfo, axis=1), pd.get_dummies(dfo)], axis=1)
df.head(3)

,summit_elev,vertical_drop,trams,fastEight,fastSixes,fastQuads,quad,triple,double,surface,...,state_Rhode Island,state_South Dakota,state_Tennessee,state_Utah,state_Vermont,state_Virginia,state_Washington,state_West Virginia,state_Wisconsin,state_Wyoming
0,2090,294,0,0.0,0,0,0,1,0,2,...,0,0,0,0,0,0,0,0,0,0
1,11100,1800,0,0.0,0,1,2,3,1,0,...,0,0,0,0,0,0,0,0,0,0
2,7800,600,0,0.0,0,0,0,1,3,1,...,0,0,0,0,0,0,0,0,0,0


## Standardize the magnitude of numeric features

**<font color='teal'> Using sklearn preprocessing, standardize the scale of the features of the dataframe except the name of the resort, which you don't need in the dataframe for modeling so it can be droppped here as well. You should  hold out your response variable(s) so that you can have their true values available for model performance review. Let's set `AdultWeekend` to the y variable as your response for scaling and modeling. Later, you will go back and consider the `AdultWeekday`, `dayOpenLastYear`, and `projectedDaysOpen`. For now leave, them in the development dataframe. </font>**

In [30]:
# hold out the response variables for later use
responseV=['AdultWeekend','AdultWeekday','daysOpenLastYear','projectedDaysOpen']
featureV=list(set(df.columns)-set(responseV))
df_response=df[responseV]
df_feature=df[featureV]
df_response.head(2)

,AdultWeekend,AdultWeekday,daysOpenLastYear,projectedDaysOpen
0,34.0,30.0,150.0,152.0
1,78.0,74.0,115.0,104.0


In [31]:
# all numeric data scaled to 0 mean and unit std
from sklearn import preprocessing
X = df_feature.values
y = df_response['AdultWeekend'].values
X_scale=preprocessing.scale(X)
y_scale=preprocessing.scale(y)

## Split into training and testing datasets

**<font color='teal'> Using sklearn model selectionm import train_test_split, and create a 75/25 split with the y = `AdultWeekend`. You will start by using the adult weekend ticket price as your response variable for modeling.</font>**

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scale, y_scale, test_size=0.25, random_state=0)

Here, you'll start the actual modeling work. First let's fit a multiple linear regression model to predict the `AdultWeekend` price.

# Guided Capstone Step 5 - Modeling

You've cleaned and prepared the datasets, so now it's time to get into the most exciting part of this process: modeling! In this exercise, you will build three different models and compare each model's performance. In the end, you will choose the best model for proving insights to Big Mountain management.  

1.   Problem Identification 

2.   Data Wrangling 
  
3.   Exploratory Data Analysis 
 
4.   Pre-processing and Training Data Development

5.   **Modeling**
  * Fit Models with Training Data Set
  * Review Model Outcomes — Iterate over additional models as needed.
  * Identify the Final Model

6.   Documentation
  * Review the Results
  * Present and share your findings - storytelling
  * Finalize Code 
  * Finalize Documentation

## Fit Models with Training Data Set

**<font color='teal'> Using sklearn, fit the model on your training dataset.</font>**

#### Model 1

In [33]:
#all first model set
from sklearn import linear_model
from sklearn.metrics import r2_score,mean_absolute_error
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)

**<font color='teal'> Predict on the testing dataset and score the model performance with the y_test set and the y-pred values. The explained variance is a measure of the variation explained by the model. This is also known as the R-squared value. </font>**

In [34]:
# prediction
y_pred=model.predict(X_test)
# R-squared value
print('r2_score for model 1: ',r2_score(y_test, y_pred))

r2_score for model 1:  0.40541086653152236


In [35]:
# mean_absolute_error
print('mean_absolute_error for model 1: ',mean_absolute_error(y_test, y_pred))

mean_absolute_error for model 1:  0.6076153018591401


## Review Model Outcomes — Iterate over additional models as needed.

**<font color='teal'> Print the intercept value from the linear model. </font>**

In [36]:
print(lm.intercept_)

-0.050096017782088206


**<font color='teal'> The intercept is the mean `AdultWeekend` price for all the resorts given the other characteristics. The addition or subtraction of each of the coefficient values in the regression are numeric adjustments applied to the intercept to provide a particular observation's value for the `AdultWeekend` resulting value. Also, because you took the time to scale your x values in the training data, you can compare each of the coefficients for the features to determine the feature's importance. Print the coefficient values from the linear model and sort in descending order to identify the top ten most important features.</font>** 

Hint: make sure to review the absolute value of the coefficients, because the adjustment may be positive or negative, but what you are looking for is the magnitude of impact on your response variable.

In [37]:
coeff=lm.coef_
print(coeff)

[-3.26031785e+08  1.63220048e-01 -1.89351668e+08 -5.24425100e+08
 -1.57968957e+12 -3.75346962e+08  1.31905556e+12  1.34528920e+11
 -6.09257674e+08 -2.66994936e+08  1.93465233e-01 -2.66994936e+08
 -5.24425100e+08 -6.07823381e+11 -1.89351668e+08 -5.35245091e-02
 -3.26031785e+08 -1.89351668e+08 -6.09257674e+08 -2.66994936e+08
 -1.84080616e-01 -1.89351668e+08 -3.26031785e+08 -3.75346962e+08
 -3.65333539e-02 -1.04430510e-01  2.94232846e+12 -1.89351668e+08
 -4.18392336e+08 -3.26031785e+08 -4.18392336e+08 -3.75346962e+08
 -2.18326808e-01  3.81469727e-02 -3.26031785e+08 -1.89351668e+08
 -6.80861094e+08 -1.89351668e+08  1.57104492e-01  2.59415062e+12
 -1.52392996e+12 -7.45591557e+11 -4.56943255e+08 -3.75346962e+08
  7.04345703e-02 -6.34357179e+08 -1.89351668e+08 -4.18392336e+08
  3.34350586e-01 -5.82716898e+08 -4.56943255e+08 -2.66994936e+08
 -7.98083157e+08 -3.26031785e+08 -1.31096467e+12]


In [39]:
df_res1=pd.DataFrame()
df_res1['feature']=df_feature.columns
df_res1['values']=abs(lm.coef_)
df_res1.sort_values(by=['values'],ascending=False)

,feature,values
26,total_chairs,2.942328e+12
39,fastSixes,2.594151e+12
4,double,1.579690e+12
40,surface,1.523930e+12
6,fastEight,1.319056e+12
54,triple,1.310965e+12
41,quad,7.455916e+11
13,fastQuads,6.078234e+11
7,trams,1.345289e+11
52,state_New York,7.980832e+08


**<font color='teal'>You should see that the top ten important features are different states. However, the state is not something the managers at the Big Mountain Resort can do anything about. Given that you care more about actionable traits associated with ticket pricing, rebuild the model without the state features and compare the results. </font>**

#### Model 2

In [40]:
# remove 'state' columns
df_feature2=df_feature.drop(df_feature.filter(regex='state').columns, axis=1)
X = df_feature2.values
# scale features
X_scale=preprocessing.scale(X)
#train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scale, y_scale, test_size=0.25, random_state=0)
# model fit
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)


In [41]:
df_feature2.head(2)

,Runs,double,fastEight,trams,Snow Making_ac,fastQuads,Cluster,SkiableTerrain_ac,LongestRun_mi,TerrainParks,total_chairs,summit_elev,yearsOpen,averageSnowfall,fastSixes,surface,quad,NightSkiing_ac,vertical_drop,triple
0,13.0,0,0.0,0,30.0,0,2,30.0,1.0,1.0,3,2090,36.0,69.0,0,2,0,30.0,294,1
1,65.0,1,0.0,0,80.0,1,1,800.0,1.2,2.0,7,11100,49.0,250.0,0,0,2,80.0,1800,3


In [42]:
# prediction
y_pred=model.predict(X_test)
# R-squared value
print('r2_score for model 2: ',r2_score(y_test, y_pred))

# mean_absolute_error
print('mean_absolute_error for model 2: ',mean_absolute_error(y_test, y_pred))

print(lm.intercept_)
print(lm.coef_)

r2_score for model 2:  0.37574746214510746
mean_absolute_error for model 2:  0.6234064456748017
-0.02648712983165972
[ 2.01426763e-01  1.08850578e-01 -1.38777878e-16 -1.38777878e-16
  2.32912676e-01  1.51445745e-01 -2.18761849e-01 -3.04132902e-01
 -4.63757952e-02 -2.42147541e-02  1.11953218e-01 -6.98017319e-02
  2.38036407e-02 -5.85577262e-02  0.00000000e+00 -1.22523611e-01
  1.39652423e-01  3.56681609e-04  3.65390715e-01  1.12889760e-01]


In [43]:
df_res2=pd.DataFrame()
df_res2['feature']=df_feature2.columns
df_res2['values']=abs(lm.coef_)
df_res2.sort_values(by=['values'],ascending=False)

,feature,values
18,vertical_drop,3.653907e-01
7,SkiableTerrain_ac,3.041329e-01
4,Snow Making_ac,2.329127e-01
6,Cluster,2.187618e-01
0,Runs,2.014268e-01
5,fastQuads,1.514457e-01
16,quad,1.396524e-01
15,surface,1.225236e-01
19,triple,1.128898e-01
10,total_chairs,1.119532e-01


**<font color='teal'> When reviewing your new model coefficients, note that `summit_elev` is now in the number two spot. From a managerial perspective, this is also difficult to change and highly correlated with `base_elev` and `vertical_drop`.  This time rebuild the model without the state features and without the `summit_elev` and without `base_elev`and compare the results. </font>**

#### Model 3

In [44]:
# remove 'summit_elev' and 'vertical_drop' columns as managers do not have handle on these
to_drop=['summit_elev','vertical_drop']
df_feature3=df_feature2.drop(to_drop,axis=1)

In [45]:
df_feature3.head(2)

,Runs,double,fastEight,trams,Snow Making_ac,fastQuads,Cluster,SkiableTerrain_ac,LongestRun_mi,TerrainParks,total_chairs,yearsOpen,averageSnowfall,fastSixes,surface,quad,NightSkiing_ac,triple
0,13.0,0,0.0,0,30.0,0,2,30.0,1.0,1.0,3,36.0,69.0,0,2,0,30.0,1
1,65.0,1,0.0,0,80.0,1,1,800.0,1.2,2.0,7,49.0,250.0,0,0,2,80.0,3


In [46]:
X = df_feature3.values
# scale features
X_scale=preprocessing.scale(X)
#train/test split
X_train, X_test, y_train, y_test = train_test_split(X_scale, y_scale, test_size=0.25, random_state=0)
# model fit
lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)


In [48]:
# prediction
y_pred=model.predict(X_test)
# R-squared value
print('r2_score for model 3: ',r2_score(y_test, y_pred))

# mean_absolute_error
print('mean_absolute_error for model 3: ',mean_absolute_error(y_test, y_pred))

print(lm.intercept_)
print(lm.coef_)

r2_score for model 3:  0.40556119880523667
mean_absolute_error for model 3:  0.6159975308725941
-0.02967625590995813
[ 3.25106175e-01  1.09363428e-01 -1.66533454e-16  8.32667268e-17
  1.94243414e-01  2.06672567e-01 -2.79375247e-01 -2.19128986e-01
  2.21986488e-02 -2.07665612e-02  1.09104537e-01  5.60047533e-02
 -7.27024118e-02 -3.46944695e-18 -1.32978393e-01  1.07308402e-01
  1.47381369e-02  1.10820850e-01]


In [49]:
df_res3=pd.DataFrame()
df_res3['feature']=df_feature3.columns
df_res3['values']=abs(lm.coef_)
df_res3.sort_values(by=['values'],ascending=False)

,feature,values
0,Runs,3.251062e-01
6,Cluster,2.793752e-01
7,SkiableTerrain_ac,2.191290e-01
5,fastQuads,2.066726e-01
4,Snow Making_ac,1.942434e-01
14,surface,1.329784e-01
17,triple,1.108209e-01
1,double,1.093634e-01
10,total_chairs,1.091045e-01
15,quad,1.073084e-01


## Identify the Final Model

**<font color='teal'> Review the model performances in the table below and choose the best model for proving insights to Big Mountain management about what features are driving ski resort lift ticket prices. Type your choice in the final markdown cell. You will explain your selection during the next step of the guided casptone. </font>**

| Model | R^2 score| Mean Absolute Error|Features Dropped|
| --- | --- | --- | --- |
| Model 1. | 0.405| 0.6076|-|
| Model 2. | 0.375|0.6234 |'state'|
| Model 3. | 0.405 |0.615 |'state','summit_elev','vertical_drop'|

Model Selection:


- Score: R^2 score maximum for model 1 and 3 but model 3 drops variable that managers can not change. Between model 2 and 3, model 3 has least Mean absolute error. Model 3 is considerable.
- Top three significant features driving the ticket price are 'Runs', 'cluster', 'skiableTerrain_Ac'. 'Cluster' categorizes all ski resorts into 3 groups. Considering this, 'Runs', 'skiableTerrain_ac', 'fastQuads'  should be the most important features to consider.